In [17]:
import pandas as pd
import numpy as np

# Scaling and Encoding
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Feature selection
from sklearn.feature_selection import SelectFromModel

# Pipiline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
# Neural Networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Accuracy metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# EDA
## Import data

In [18]:
train = pd.read_csv('churn-bigml-80.csv', encoding='utf-8')
test = pd.read_csv('churn-bigml-20.csv', encoding='utf-8')

## Descriptive analysis

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int64  
 2   Area code               2666 non-null   int64  
 3   International plan      2666 non-null   object 
 4   Voice mail plan         2666 non-null   object 
 5   Number vmail messages   2666 non-null   int64  
 6   Total day minutes       2666 non-null   float64
 7   Total day calls         2666 non-null   int64  
 8   Total day charge        2666 non-null   float64
 9   Total eve minutes       2666 non-null   float64
 10  Total eve calls         2666 non-null   int64  
 11  Total eve charge        2666 non-null   float64
 12  Total night minutes     2666 non-null   float64
 13  Total night calls       2666 non-null   int64  
 14  Total night charge      2666 non-null   

In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   667 non-null    object 
 1   Account length          667 non-null    int64  
 2   Area code               667 non-null    int64  
 3   International plan      667 non-null    object 
 4   Voice mail plan         667 non-null    object 
 5   Number vmail messages   667 non-null    int64  
 6   Total day minutes       667 non-null    float64
 7   Total day calls         667 non-null    int64  
 8   Total day charge        667 non-null    float64
 9   Total eve minutes       667 non-null    float64
 10  Total eve calls         667 non-null    int64  
 11  Total eve charge        667 non-null    float64
 12  Total night minutes     667 non-null    float64
 13  Total night calls       667 non-null    int64  
 14  Total night charge      667 non-null    fl

* There is no null values
* For further analysis, categorical columns should be converted into numerical.
    * For column 3,4: Yes=1, No=0
    * For `State`, `Area code` columns: One Hot encoding *(this step will be combined with Scaling later)*
## Cleaning
### Remove outliers
Z-score

In [22]:
def remove_outliers(df, threshold=2):
    numeric_cols = list(df.columns[1:2]) + list(df.columns[5:19])

    for col in numeric_cols:
        col_zscore = (df[col] - df[col].mean()) / df[col].std()
        df = df[(col_zscore < threshold) & (col_zscore > -threshold)]
        
    return df

In [23]:
train = remove_outliers(train)
test = remove_outliers(test)

print(len(train))
print(len(test))

1384
342


In [24]:
train.describe()

,Account length,Area code,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
count,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000
mean,99.627890,438.362717,6.170520,180.413367,99.975434,30.670838,199.311850,100.104769,16.941756,202.110332,100.252890,9.095051,10.348699,4.156792,2.794675,1.467486
std,34.404755,43.064061,11.549064,44.582349,17.483660,7.578940,42.083341,17.617898,3.577088,41.757612,17.088801,1.879194,2.164196,1.928396,0.584285,1.139109
min,22.000000,408.000000,0.000000,86.100000,61.000000,14.640000,109.900000,61.000000,9.340000,112.200000,62.000000,5.050000,5.700000,1.000000,1.540000,0.000000
25%,74.000000,408.000000,0.000000,146.725000,87.750000,24.947500,167.275000,87.000000,14.217500,170.300000,88.000000,7.660000,8.800000,3.000000,2.380000,1.000000
50%,99.000000,415.000000,0.000000,179.400000,100.000000,30.500000,200.250000,100.000000,17.025000,202.050000,101.000000,9.090000,10.400000,4.000000,2.810000,1.000000
75%,124.000000,510.000000,0.000000,213.425000,113.000000,36.285000,230.400000,113.000000,19.580000,233.700000,113.000000,10.520000,12.000000,5.000000,3.240000,2.000000
max,179.000000,510.000000,35.000000,275.200000,139.000000,46.780000,289.300000,140.000000,24.590000,290.000000,139.000000,13.050000,14.900000,9.000000,4.020000,4.000000


In [27]:
test.describe()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
count,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000
mean,102.789474,434.865497,0.087719,0.242690,6.657895,181.786842,101.885965,30.904415,204.093275,101.602339,17.347953,198.525731,99.961988,8.933626,10.529240,4.324561,2.843421,1.459064
std,35.651903,41.002675,0.283301,0.429337,12.122417,45.590944,18.036665,7.750406,39.087594,16.452894,3.322582,40.208674,18.032108,1.809633,2.182419,2.062778,0.589443,1.089710
min,25.000000,408.000000,0.000000,0.000000,0.000000,81.900000,62.000000,13.920000,120.700000,63.000000,10.260000,115.900000,60.000000,5.220000,5.700000,1.000000,1.540000,0.000000
25%,79.000000,408.000000,0.000000,0.000000,0.000000,149.550000,88.250000,25.422500,174.200000,89.000000,14.807500,167.825000,87.000000,7.555000,8.900000,3.000000,2.400000,1.000000
50%,102.000000,415.000000,0.000000,0.000000,0.000000,180.300000,101.000000,30.650000,202.900000,102.000000,17.250000,200.200000,100.000000,9.010000,10.600000,4.000000,2.860000,1.000000
75%,127.000000,415.000000,0.000000,0.000000,0.000000,216.075000,115.000000,36.730000,233.675000,113.000000,19.860000,228.550000,112.750000,10.287500,12.100000,6.000000,3.270000,2.000000
max,183.000000,510.000000,1.000000,1.000000,36.000000,281.400000,141.000000,47.840000,287.700000,138.000000,24.450000,283.200000,139.000000,12.740000,15.200000,9.000000,4.100000,4.000000


### Converting data types
For column 3,4: Yes=1, No=0

In [25]:
def replace_y_n(df, column_name):
    if len(df[column_name].unique()) == 2: # to ensure there is no mispelling word
        df[column_name].replace({'Yes':1, 'No':0}, inplace=True)
        print(df[column_name].unique())

In [26]:
replace_y_n(train, 'International plan')
replace_y_n(test, 'International plan')
replace_y_n(train,'Voice mail plan')
replace_y_n(test, 'Voice mail plan')

[0 1]
[0 1]
[1 0]
[0 1]


## Prepare variables

In [28]:
X_train = train.drop('Churn', axis=1)
X_test = test.drop('Churn', axis=1)

y_train = train['Churn']
y_test = test['Churn']

# Preprocessing, Feature Selection & Main model
Here, I built a pipeline that contains:
* Preprocessing for numerical data: Standard scaling
* Preprocessing for categorical data: One-hot encoding
* Feature Selection
* Classifition model

## Preprocessing
Scaling and Encoding

In [29]:
# Define numerical and categorical columns
list_of_num_cols = list(train.columns[1:2]) + list(train.columns[5:19])
list_of_cat_cols = list(train.columns[0:1]) + list(train.columns[5:6])

In [30]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(transformers= [('num', numerical_transformer, list_of_num_cols)
                                                ,('cat', categorical_transformer, list_of_cat_cols)
                                                ]
                                )

## Feature Selection & Main model
As there are several estimators in `SelectFromModel`, I'd consider all of those
* Logistic Regression
* SVM
* Decision Tree
* Random Forest
* Gradient Boosting

In [31]:
def models(feature_selection, classifier):
    # define Feature Selection and Classifier
    feature_selector = SelectFromModel(feature_selection)
    model = classifier
    
    
    # Combine into Pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('feature_selection', feature_selector),
                           ('model', model)])
    
    
    # Train model
    pipeline.fit(X_train,y_train)
    
        
    # Accuracy checks
    y_pred = pipeline.predict(X_test)

    print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
    print(f'Precision: {precision_score(y_test, y_pred):.2f}')
    print(f'Recall: {recall_score(y_test, y_pred):.2f}')
    print(f'F1 Score: {f1_score(y_test, y_pred):.2f}')
    print('---')

In [32]:
list_of_models = [LogisticRegression()
                  , LinearSVC()
                  , DecisionTreeClassifier()
                  , RandomForestClassifier()
                  , GradientBoostingClassifier()
                  ]

In [33]:
for classifier in list_of_models:
    for feature_selector in list_of_models:
        models(feature_selection=feature_selector, classifier=classifier)

Accuracy: 0.90
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
---
Accuracy: 0.91
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
---


c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.91
Precision: 0.67
Recall: 0.06
F1 Score: 0.11
---
Accuracy: 0.91
Precision: 0.50
Recall: 0.03
F1 Score: 0.06
---
Accuracy: 0.91
Precision: 0.50
Recall: 0.03
F1 Score: 0.06
---
Accuracy: 0.91
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
---
Accuracy: 0.91
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
---


c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: Th

Accuracy: 0.91
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
---


...

Accuracy: 0.93
Precision: 0.89
Recall: 0.25
F1 Score: 0.39
---
Accuracy: 0.92
Precision: 0.88
Recall: 0.22
F1 Score: 0.35
---
Accuracy: 0.92
Precision: 0.88
Recall: 0.22
F1 Score: 0.35
---
Accuracy: 0.90
Precision: 0.33
Recall: 0.03
F1 Score: 0.06
---


c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.91
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
---


c:\Users\ASUS FLIP 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.92
Precision: 0.60
Recall: 0.28
F1 Score: 0.38
---
Accuracy: 0.91
Precision: 0.57
Recall: 0.25
F1 Score: 0.35
---
Accuracy: 0.91
Precision: 0.57
Recall: 0.25
F1 Score: 0.35
---
